# Proyecto DL

### Librerias

In [ ]:
!pip install rouge
!pip install sacrebleu

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
from rouge import Rouge
import sacrebleu

### Modelo y función de translate

https://huggingface.co/Helsinki-NLP/opus-mt-es-en

In [ ]:
model_name = 'Helsinki-NLP/opus-mt-es-en'

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
#model

In [ ]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    translated = model.generate(**inputs)

    translation = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translation

In [ ]:
input = "He aprendido mucho sobre modelos de lenguaje en este proyecto"
reference = "I have learned a lot about language models in this project"
candidate = translate(input)

print("Candidate:", candidate)
print("Referencia:", reference)

### Metrica Rouge

https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499

https://github.com/pltrdy/rouge

R: reference

C: candidate


In [ ]:
rouge = Rouge()
scores = rouge.get_scores(candidate, reference)
scores

### Metrica bleu

https://aclanthology.org/P02-1040.pdf

In [ ]:
bleu_score = sacrebleu.corpus_bleu([candidate], [[reference]])
bleu_score.score

In [ ]:
# import sacrebleu

# # Función para calcular BLEU Score
# def calculate_bleu(reference_texts, candidate_texts):
#     bleu_score = sacrebleu.corpus_bleu(candidate_texts, [reference_texts])
#     return bleu_score.score

# # Extracción de las columnas necesarias
# reference_texts = data['reference'].tolist()
# candidate_texts = data['candidate'].tolist()

# # Cálculo del BLEU Score
# bleu_score = calculate_bleu(reference_texts, candidate_texts)
# print(f"BLEU Score: {bleu_score}")

### Dataset

https://www.dropbox.com/scl/fi/xtvls58le65vxjfkmuv9x/Sentence-pairs-in-Spanish-English-2024-07-03.tsv?rlkey=3la4vmua8d5f5ltdknfhmcmyt&st=6cdj4k63&dl=0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Nose que significa ese number, en vola id de la traduccion tb?

In [ ]:
column_names = ['id_reference', 'reference', 'number', 'candidate']

data = pd.read_csv('/content/drive/MyDrive/Sentence pairs in Spanish-English - 2024-07-03.tsv', sep='\t', names=column_names, header=None)

In [ ]:
data.head()
# data.describe()

### NER(Named-entity recognition)

https://github.com/marcelomendoza/IIC3670/blob/main/codes/8%20-%20SEQUENCE%20LABELING%2C%20POS%20TAGGING%20Y%20NER.ipynb